In [255]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,root_mean_squared_error
from xgboost import XGBRegressor
import numpy as np
import joblib

In [256]:
DATA_PATH = '../data/processed/procssed.csv'
OUTPUT_MODEL_PATH = '../output/models/model.joblib'
df = pd.read_csv(DATA_PATH)


In [257]:
X = df.drop("Price", axis=1)
y = df["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [258]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [259]:
xgb = XGBRegressor(n_estimators=700,learning_rate=0.04,max_depth=6,subsample=0.8,colsample_bytree=0.8,reg_lambda=2.0,objective="reg:squarederror",random_state=42)
xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.04, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=700,
             n_jobs=None, num_parallel_tree=None, ...)

In [260]:
lin = LinearRegression()
lin.fit(X_train, y_train)


LinearRegression()

In [261]:
poly = PolynomialFeatures(degree=2,include_bias=False)
xtr = poly.fit_transform(X_train)
xts = poly.transform(X_test)
pr = Ridge(alpha=1.0)
# pr.fit(xtr,xts)


In [262]:
import numpy as np

def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)
    
    metrics = {
        'MSE': mean_squared_error(y_test, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred),
        "absolute":mean_absolute_error(y_test, y_pred),
        "root":root_mean_squared_error(y_test, y_pred)
    }
    
    return metrics, y_pred

def print_metrics(metrics):

    print("نتائج تقييم النموذج:")
    print(f"الدقة (MSE): {metrics['MSE']:.4f}")
    print(f"الدقة (rmse): {metrics['rmse']:.4f}")
    print(f"الدقة (R2): {metrics['R2']:.4f}")
    print(f"الدقة (absolute): {metrics['absolute']:.4f}")
    print(f"الدقة (root): {metrics['root']:.4f}")


In [263]:
metrics, y_pred = evaluate_model(rf,X_test,y_test)
print_metrics(metrics)

نتائج تقييم النموذج:
الدقة (MSE): 437048840.4214
الدقة (rmse): 20905.7131
الدقة (R2): 0.8525
الدقة (absolute): 9695.3617
الدقة (root): 20905.7131


In [264]:
metrics, y_pred = evaluate_model(xgb,X_test,y_test)
print_metrics(metrics)

نتائج تقييم النموذج:
الدقة (MSE): 352100565.9052
الدقة (rmse): 18764.3429
الدقة (R2): 0.8812
الدقة (absolute): 9552.3919
الدقة (root): 18764.3429


In [265]:
metrics, y_pred = evaluate_model(lin,X_test,y_test)
print_metrics(metrics)

نتائج تقييم النموذج:
الدقة (MSE): 1381949238.0685
الدقة (rmse): 37174.5778
الدقة (R2): 0.5337
الدقة (absolute): 24431.4865
الدقة (root): 37174.5778


In [266]:
metrics, y_pred = evaluate_model(pr,X_test,y_test)
print_metrics(metrics)

NotFittedError: This Ridge instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
joblib.dump(rf,OUTPUT_MODEL_PATH)

['../output/models/model.joblib']